In [ ]:
from datamodule import Sec_Data_Module
from model import Sec_Classifier
from config import config
import os
import pytorch_lightning as pl

In [ ]:
!export CUDA_VISIBLE_DEVICES=2

In [ ]:
config

In [ ]:
model_list = ['xlm-roberta-base', 'bert-base-uncased', 'jackaduma/SecRoBERTa']
for model_name in model_list:
    config['model_name'] = model_name
    epoch_num = config['n_epochs']

    print(f"{epoch_num}epoch train started!")

    model_name = config['model_name'].split('/')[-1]
    data_path = f'../{model_name}_dataset/'
    train_path = data_path + 'train.pkl'
    val_path = data_path + 'val.pkl'

    data_module = Sec_Data_Module(train_path, val_path, batch_size=config['batch_size'], model_name=config['model_name'], max_token_len=512)
    data_module.setup()
    config['train_size'] = len(data_module.train_dataloader())
    model = Sec_Classifier(config)

    trainer = pl.Trainer(max_epochs=config['n_epochs'], num_sanity_val_steps=-1, accelerator="gpu", devices=1)
    # trainer = pl.Trainer(max_epochs=config['n_epochs'], num_sanity_val_steps=-1, accelerator="gpu", devices=-1, strategy="ddp_find_unused_parameters_true")
    # trainer = pl.Trainer(max_epochs=config['n_epochs'], accelerator="gpu", devices=4, strategy="dp")
    trainer.fit(model, data_module)

    if not os.path.exists(f'{model_name}_ckpt'):
        os.mkdir(f'{model_name}_ckpt')

    model_save_dir = f'./{model_name}_ckpt/'

    # trainer.save_checkpoint(model_save_dir, f'{model_name}_{epoch_num}Model.ckpt')

    # predictions = trainer.predict(model, data_module)
    # torch.save(model.state_dict(), model_save_dir+f'{model_name}_{epoch_num}Model.pth')

    # with open(pred_path+f'{sample_num}SamplePredictions.pickle', 'wb') as fw:
    #     pickle.dump(predictions, fw)

    print(f"{epoch_num}epoch train completed!")